### Web scrapping exercise

## Webscraping

In this exercise, you'll practice using BeautifulSoup to parse the content of a web page. The page that you'll be scraping, https://realpython.github.io/fake-jobs/, contains job listings. Your job is to extract the data on each job and convert into a pandas DataFrame.

In [6]:
import requests
from bs4 import BeautifulSoup

## 1. Start by performing a GET request on the url above and convert the response into a BeautifulSoup object.  

In [4]:
URL = "https://realpython.github.io/fake-jobs/"

headers = {
    "User-Agent": "MyPythonScraper"
}

response = requests.get(URL, headers=headers)
response.status_code #confirm it loaded correctly

#convert it into a soup object
soup = BeautifulSoup(response.text)

200

### 1a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title.  

In [14]:
#insepcted soup (pretify) --> jpob titles seem to be under <h2> class
job_pos_title= soup.find('h2').text

'Senior Python Developer'

### 1b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list.  

In [17]:
job_title_list= soup.findAll('h2')
len(job_title_list)

100

In [61]:
soup.find('div').get('column is half').text

AttributeError: 'NoneType' object has no attribute 'text'

### 1c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end.  

THe organization => section > container ('div') > card > card content or media
* Each card has a MEDIA section and a CONTENT section
* content = location, date
* media = job posting, company

In [128]:
import pandas as pd
from io import StringIO

In [105]:
all_card_content= soup.findAll('div', attrs={'class': 'content'})

In [113]:
all_card_media =soup.findAll('div', attrs={'class': 'media'})
#soup.findAll('div', attrs={"class": "media"})[0].find('h2').text

In [162]:
## iterate between every card content to get the title, company, location, and dates 
location_list=[]
time_list=[]
title_list=[]
company_list=[]
for card in all_card_content:
    location = card.find('p', attrs={"class":"location"}).text.replace('\n', '').lstrip('')
    location_list.append(location)
    time_posted= card.find('time').get('datetime')
    time_list.append(time_posted)

for media in all_card_media:
    title= media.find('h2').text
    title_list.append(title)
    company= media.find('h3').text
    company_list.append(company)

### 1d. Take the lists that you have created and combine them into a pandas DataFrame. 

In [163]:
job_card_df= pd.DataFrame({"job title": title_list, "company": company_list, "location": location_list, "time posted": time_list})
job_card_df

,job title,company,location,time posted
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08
...,...,...,...,...
95,Museum/gallery exhibitions officer,"Nguyen, Yoder and Petty","Lake Abigail, AE",2021-04-08
96,"Radiographer, diagnostic",Holder LLC,"Jacobshire, AP",2021-04-08
97,Database administrator,Yates-Ferguson,"Port Susan, AE",2021-04-08
98,Furniture designer,Ortega-Lawrence,"North Tiffany, AA",2021-04-08


### 2. Next, add a column that contains the url for the "Apply" button. Try this in two ways. 

### 2a. First, use the BeautifulSoup find_all method to extract the urls.  

In [185]:
soup.findAll('a', attrs={"class":"card-footer-item"})

[<a class="card-footer-item" href="https://www.realpython.com" target="_blank">Learn</a>,
 <a class="card-footer-item" href="https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html" target="_blank">Apply</a>,
 <a class="card-footer-item" href="https://www.realpython.com" target="_blank">Learn</a>,
 <a class="card-footer-item" href="https://realpython.github.io/fake-jobs/jobs/energy-engineer-1.html" target="_blank">Apply</a>,
 <a class="card-footer-item" href="https://www.realpython.com" target="_blank">Learn</a>,
 <a class="card-footer-item" href="https://realpython.github.io/fake-jobs/jobs/legal-executive-2.html" target="_blank">Apply</a>,
 <a class="card-footer-item" href="https://www.realpython.com" target="_blank">Learn</a>,
 <a class="card-footer-item" href="https://realpython.github.io/fake-jobs/jobs/fitness-centre-manager-3.html" target="_blank">Apply</a>,
 <a class="card-footer-item" href="https://www.realpython.com" target="_blank">Learn</a>,
 <a class="card

<div class="content">
<p class="location">
        Stewartbury, AA
      </p>
<p class="is-small has-text-grey">
<time datetime="2021-04-08">2021-04-08</time>
</p>
</div>

    b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.

    
3. Finally, we want to get the job description text for each job.  
    a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.  
    b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".  
    c. Use the [.apply method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) on the url column you created above to retrieve the description text for all of the jobs.